In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config_2 import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mbandaka,CD,0.0487,18.2603,70.52,97,100,1.28,light rain
1,1,Kodiak,US,57.7900,-152.4072,63.91,25,0,10.36,clear sky
2,2,Mirnyy,RU,62.5353,113.9611,55.24,81,61,13.00,broken clouds
3,3,Dikson,RU,73.5069,80.5464,30.85,95,97,15.28,overcast clouds
4,4,Aguimes,ES,27.9054,-15.4461,66.83,73,20,18.41,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mbandaka,CD,0.0487,18.2603,70.52,97,100,1.28,light rain
12,12,Rikitea,PF,-23.1203,-134.9692,76.84,76,100,15.21,light rain
18,18,Mount Isa,AU,-20.7333,139.5000,78.57,44,2,10.36,clear sky
23,23,Pacific Grove,US,36.6177,-121.9166,75.54,66,0,10.00,clear sky
27,27,Georgetown,MY,5.4112,100.3354,84.13,84,20,2.30,few clouds
29,29,Hauterive,FR,46.0901,3.4459,70.56,85,97,5.66,overcast clouds
37,37,Lompoc,US,34.6391,-120.4579,84.02,63,0,10.36,clear sky
39,39,Kapaa,US,22.0752,-159.3190,75.18,94,100,21.85,heavy intensity rain
41,41,Atuona,PF,-9.8000,-139.0333,78.46,69,59,13.80,broken clouds
43,43,Makakilo City,US,21.3469,-158.0858,77.32,83,100,11.50,light rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                205
City                   205
Country                205
Lat                    205
Lng                    205
Max Temp               205
Humidity               205
Cloudiness             205
Wind Speed             205
Current Description    205
dtype: int64

In [6]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Mbandaka,CD,70.52,light rain,0.0487,18.2603,
12,Rikitea,PF,76.84,light rain,-23.1203,-134.9692,
18,Mount Isa,AU,78.57,clear sky,-20.7333,139.5000,
23,Pacific Grove,US,75.54,clear sky,36.6177,-121.9166,
27,Georgetown,MY,84.13,few clouds,5.4112,100.3354,
29,Hauterive,FR,70.56,overcast clouds,46.0901,3.4459,
37,Lompoc,US,84.02,clear sky,34.6391,-120.4579,
39,Kapaa,US,75.18,heavy intensity rain,22.0752,-159.3190,
41,Atuona,PF,78.46,broken clouds,-9.8000,-139.0333,
43,Makakilo City,US,77.32,light rain,21.3469,-158.0858,


In [7]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [8]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
# clean_hotel_df.head()

In [9]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [10]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [11]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))